# Part 1

In [254]:
#import all requred libraries
import requests
import pandas as pd
import numpy as np
import random
from bs4 import BeautifulSoup
import requests
import re
import geocoder

Get the url text data and create a soup object, then fnd anything with tag table in the html data

In [255]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text 
soup = BeautifulSoup(data,"html5lib")
table = soup.find('table')

Create an empty dataframe, then go row by row in each table and add values to the table_result 

In [256]:
table_result = pd.DataFrame(columns=["Postal Code", "Borough",'Neighbourhood'])

for row in table.tbody.find_all("tr"):
    col = row.find_all("td")
    if (col != []):
        PostalCode = col[0].text
        Borough = col[1].text
        Neighbourhood = col[2].text
        table_result = table_result.append({"Postal Code":PostalCode, "Borough":Borough,'Neighbourhood':Neighbourhood}, ignore_index=True)

Filter the dataframe to rows where Borough is not 'Not assigned'

In [257]:
table_result = table_result[table_result['Borough']!= 'Not assigned\n']

Remove all '\n' from the dataframe

In [258]:
table_result = table_result.replace(r'\n',' ', regex=True) 

Groupy the dataframe by postal code and borough using a comma, reset index and add empty columns for futher use

In [259]:
table_result = table_result.groupby(['Postal Code','Borough'], sort=False).agg( ', '.join)
table_result = table_result.reset_index()
table_result['Latitude']= ''
table_result['Longitude']= ''


Change each not assigned Borough to the neighbourhood

In [260]:
for index, each in enumerate(table_result['Borough']):
    if each == 'Not assigned':
        table_result.loc[index,'Borough'] = table_result['Neighbourhood'][index]

Remove all spaces in the Postal Code column

In [261]:
table_result['Postal Code'] = table_result['Postal Code'].str.replace(' ', '')
table_result

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,,
1,M4A,North York,Victoria Village,,
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",,
3,M6A,North York,"Lawrence Manor, Lawrence Heights",,
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",,
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",,
99,M4Y,Downtown Toronto,Church and Wellesley,,
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",,
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",,


Download the geospacial data to a 'long_lat_data' dataframe, and set the postal code as index

# Part 2

In [262]:
long_lat = 'https://cocl.us/Geospatial_data'
!wget -q -O 'toronto_m.geospatial_data.csv' long_lat
long_lat_data = pd.read_csv(long_lat).set_index("Postal Code")
long_lat_data.head()

'wget' is not recognized as an internal or external command,
operable program or batch file.


,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


Merge the imported geospacial data with main dataframe and save it as final_df, then clean the dataframe.

In [263]:
final_df = pd.merge(long_lat_data, table_result, on='Postal Code')
final_df = final_df[['Postal Code', 'Borough', 'Neighbourhood', 'Latitude_x', 'Longitude_x']]
final_df.rename(columns={'Latitude_x':'Latitude'}, inplace=True)
final_df.rename(columns={'Longitude_x':'Longitude'}, inplace=True)
final_df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


# Part 3

In [264]:
#import appropriate libraries
from geopy.geocoders import Nominatim
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Find coordinates of toronto

In [265]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Filter the final_df to boroughs that contain toronto and reset index

In [266]:
clustering_df=final_df[final_df['Borough'].str.contains('Toronto')]
clustering_df=clustering_df.reset_index(drop=True)
clustering_df.shape

(39, 5)

Remove spaces in borough names and change names to numerical values for further analysis.

In [267]:
clustering_df['Borough'] = clustering_df['Borough'].str.replace(' ', '')
clustering_df['Label']=clustering_df['Borough'].replace(to_replace=['DowntownToronto','CentralToronto','WestToronto','EastToronto'],value=[1,2,3,4],inplace=False)
clustering_df['Borough'].value_counts()
clustering_df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Label
0,M4E,EastToronto,The Beaches,43.676357,-79.293031,4
1,M4K,EastToronto,"The Danforth West, Riverdale",43.679557,-79.352188,4
2,M4L,EastToronto,"India Bazaar, The Beaches West",43.668999,-79.315572,4
3,M4M,EastToronto,Studio District,43.659526,-79.340923,4
4,M4N,CentralToronto,Lawrence Park,43.728020,-79.388790,2


Set amount clusters as unique values of labels. Create a map with coordinates of Toronto.
Set rainbow colour ranges

In [268]:
clusters=len(clustering_df.Label.unique())
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
x = np.arange(clusters)
ys = [i + x + (i*x)**2 for i in range(clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

Add markers to the map by iterating through the dataframe.

In [271]:
markers_colors = []
for lat, lon, cluster in zip(clustering_df['Latitude'], clustering_df['Longitude'], clustering_df['Label']):
    label = folium.Popup(str(clustering_df['Borough']) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_toronto)

Print the map

In [272]:
map_toronto